<a href="https://colab.research.google.com/github/VitaliyShein/Clusterizer/blob/main/ClusterizerWithLog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%python -m pip install pip<24.1

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
%pip install textract
import textract

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
%pip install langdetect

  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=42babd49e5cad0bc4f6c6559f16d93a60b8d97ae0845c489d308c8a8a292f1ce
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
import numpy as np
import shutil
import os
from sklearn.preprocessing import Normalizer

In [77]:
import logging

# Настройка логирования
logging.basicConfig(
    level=logging.DEBUG,
    filemode='w',
    handler=logging.StreamHandler()
)
logger = logging.getLogger(__name__)

#logger.setLevel(logging.DEBUG)

In [78]:
from google.colab import drive
logger.info("Монтирование Google Drive")
drive.mount('/content/drive', force_remount=True)

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

Mounted at /content/drive


In [79]:
file_handler = logging.FileHandler('/content/drive/MyDrive/Project0/clusterizer.log', mode='w')
formatter = logging.Formatter("[%(name)s] [%(asctime)s] [%(levelname)s]: %(message)s")
file_handler.setFormatter(formatter)

logger.addHandler(file_handler)

In [80]:
logger.debug("debug")

--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
 

In [81]:
"""Загрузка BERT"""

from transformers import BertTokenizer, pipeline
import torch

# Для английского
en_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
logger.info("Загружен суммаризатор для английского языка")

# Для русского
ru_summarizer = pipeline(
    "summarization",
    model="IlyaGusev/rut5_base_sum_gazeta",
    tokenizer="IlyaGusev/mbart_ru_sum_gazeta"
)
logger.info("Загружен суммаризатор для русского языка")

def summarize_text(text, summarizer):
    """Генерация краткого описания"""
    try:
        logger.info(f"Суммаризация текста длиной {len(text)} символов")
        summary = summarizer(
          text[:2000] if len(text) > 20 else text,
          max_length=15,
          min_length=3,
          length_penalty=0.2  # Делаем суммаризацию короче
        )
        logger.debug(f"Результат суммаризации: {summary[0]['summary_text']}")
        return summary[0]['summary_text']
    except Exception as e:
        logger.error(f"Ошибка суммаризации: {e}")
        return 0


Device set to use cpu
--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/usr/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 107] Transport endpoint is not connected
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line

In [82]:
# -*- coding: utf-8 -*-
"""Clusterizer3.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1BmqVrNRektDk563onVVSwqvl6Qao998j
"""



# Commented out IPython magic to ensure Python compatibility.
# %pip install textract


# Commented out IPython magic to ensure Python compatibility.
# %%python -m pip install pip<24.1

# Commented out IPython magic to ensure Python compatibility.
# %pip install langdetect



def get_embeddings(model_name='all-MiniLM-L6-v2'):
    """
    Преобразует список текстов в эмбеддинги

    Args:
        texts (list): Список текстовых строк
        model_name (str): Название предобученной модели (по умолчанию 'all-MiniLM-L6-v2')

    Returns:
        np.array: Массив эмбеддингов размером (n_texts, embedding_dim)
    """
    texts = []
    model = SentenceTransformer(model_name)
    logger.info(f"Загружена модель для эмбеддингов: {model_name}")

    # Путь к папке с файлами
    folder_path = "/content/drive/MyDrive/TrainData"
    logger.info(f"Чтение файлов из папки: {folder_path}")

    # Поддерживаемые форматы
    supported_extensions = ['.pdf', '.docx', '.txt', '.pptx', '.jpg', '.png', '.jpeg']

    # Проходим по всем файлам в папке
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Проверяем расширение файла
        if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in supported_extensions):
            try:
                logger.info(f"Обработка файла: {filename}")
                text = textract.process(file_path).decode('utf-8')
                texts.append(text)
                logger.debug(f"Успешно извлечен текст из файла {filename}, длина текста: {len(text)} символов")
            except Exception as e:
                logger.error(f"Ошибка при обработке файла {filename}: {str(e)}")

    if not texts:
        logger.warning("Не найдено ни одного файла с поддерживаемым форматом")
        return None

    logger.info(f"Всего обработано файлов: {len(texts)}")
    embeddings = model.encode(texts, convert_to_numpy=True)
    logger.info(f"Созданы эмбеддинги для {len(texts)} текстов, размерность: {embeddings.shape}")

    # Нормализация L2 (делает длину векторов = 1)
    normalizer = Normalizer(norm='l2')
    embeddings_normalized = normalizer.transform(embeddings)
    logger.debug(f"Выполнена нормализация эмбеддингов")

    # Проверка
    norm_check = np.linalg.norm(embeddings_normalized[0])
    logger.debug(f"Проверка нормализации: норма первого вектора = {norm_check}")
    return embeddings_normalized

def get_ONE_embedding(text, model_name='all-MiniLM-L6-v2'):
    """
    Преобразует текст в эмбеддинг

    Args:
        text (str): Текстовая строка
        model_name (str): Название предобученной модели

    Returns:
        np.array: Нормализованный эмбеддинг
    """
    logger.info(f"Создание эмбеддинга для текста длиной {len(text)} символов")
    model = SentenceTransformer(model_name)
    ONE_embedding = model.encode([text], convert_to_numpy=True)

    # Нормализация L2 (делает длину векторов = 1)
    normalizer = Normalizer(norm='l2')
    ONE_embedding_normalized = normalizer.transform(ONE_embedding)

    # Проверка
    norm_check = np.linalg.norm(ONE_embedding_normalized[0])
    logger.debug(f"Проверка нормализации: норма вектора = {norm_check}")

    return ONE_embedding_normalized

def train_cluster_model(embeddings, n_clusters=3, random_state=42):
    """
    Обучает модель кластеризации на эмбеддингах

    Args:
        embeddings (np.array): Массив эмбеддингов
        n_clusters (int): Количество кластеров
        random_state (int): Для воспроизводимости

    Returns:
        KMeans: Обученная модель K-Means
    """
    if embeddings is None:
        logger.error("Не переданы эмбеддинги для обучения")
        raise ValueError("Embeddings cannot be None")

    logger.info(f"Начало обучения модели KMeans с {n_clusters} кластерами")
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
    kmeans.fit(embeddings)
    logger.info(f"Модель KMeans успешно обучена, центроиды: {kmeans.cluster_centers_.shape}")
    return kmeans

def predict_cluster(text, kmeans_model):
    """
    Предсказывает кластер для нового текста

    Args:
        text (str): Текст для классификации
        kmeans_model: Обученная модель кластеризации

    Returns:
        int: Номер кластера (0, 1, 2, ...)
    """
    logger.info(f"Предсказание кластера для текста длиной {len(text)} символов")
    embedding = get_ONE_embedding(text)
    cluster = kmeans_model.predict(embedding)
    logger.info(f"Предсказан кластер {cluster[0]} для текста")
    return cluster[0]


from langdetect import detect

def detect_language(text):
    try:
        lang = detect(text)
        logger.debug(f"Определен язык текста: {lang}")
        return lang
    except Exception as e:
        logger.error(f"Ошибка определения языка: {e}")
        return "unknown"

# Путь к папке с файлами
folder_path = "/content/drive/MyDrive/InputData"
logger.info(f"Работа с папкой: {folder_path}")

# Поддерживаемые форматы
supported_extensions = ['.pdf', '.docx', '.txt', '.pptx', '.jpg', '.png', '.jpeg']

# Проходим по всем файлам в папке
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Проверяем расширение файла
    if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in supported_extensions):
        try:
            logger.info(f"Начало обработки файла: {filename}")

            # Извлекаем текст
            text = textract.process(file_path).decode('utf-8')
            logger.debug(f"Извлечен текст из файла {filename}, длина: {len(text)} символов")

            # Определяем язык
            text_lang = detect_language(text)
            logger.info(f"Для файла {filename} определен язык: {text_lang}")

            # Определение краткого описания текста
            if text_lang == "ru":
                text_description = summarize_text(text, ru_summarizer)
            elif text_lang == "en":
                text_description = summarize_text(text, en_summarizer)
            else:
                text_description = "unknown_language"
                logger.warning(f"Неизвестный язык для файла {filename}")

            logger.info(f"Создано описание для файла {filename}: {text_description}")

            # Переименование
            new_path = os.path.join(folder_path, text_description)
            shutil.move(file_path, new_path)
            logger.info(f"Файл {filename} переименован в {text_description}")

        except Exception as e:
            logger.error(f"Ошибка при обработке файла {filename}: {str(e)}")

embeddings = get_embeddings()
kmeans_model = train_cluster_model(embeddings)

def move_to_folder(cluster, filename):
    """Перемещает файл в папку соответствующего кластера"""
    directory = "/content/drive/MyDrive/InputData"
    logger.info(f"Перемещение файла {filename} в кластер {cluster}")

    destination_path = f"/content/drive/MyDrive/{cluster}"
    logger.debug(f"Целевая папка: {destination_path}")

    # Создать папку, если её нет
    if not os.path.exists(destination_path):
        logger.debug(f"Создание папки {destination_path}")
        os.makedirs(destination_path)

    begPath = os.path.join(directory, filename)
    try:
        shutil.copy(begPath, destination_path)
        logger.info(f"Файл {filename} успешно скопирован в {destination_path}")
    except Exception as e:
        logger.error(f"Ошибка при копировании файла {filename}: {str(e)}")

# Путь к папке с файлами
folder_path = "/content/drive/MyDrive/InputData"
logger.info(f"Кластеризация файлов в папке: {folder_path}")

# Поддерживаемые форматы
supported_extensions = ['.pdf', '.docx', '.txt', '.pptx', '.jpg', '.png', '.jpeg']

# Проходим по всем файлам в папке
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    # Проверяем расширение файла
    if os.path.isfile(file_path) and any(file_path.lower().endswith(ext) for ext in supported_extensions):
        try:
            logger.info(f"Обработка файла для кластеризации: {filename}")

            # Извлекаем текст
            text = textract.process(file_path).decode('utf-8')
            logger.debug(f"Извлечен текст из файла {filename}, длина: {len(text)} символов")

            # Находим в какой кластер попадает текст
            cluster = predict_cluster(text, kmeans_model)
            logger.info(f"Файл {filename} отнесен к кластеру {cluster}")

            # Перемещаем файл в папку соответствующего кластера
            move_to_folder(cluster, filename)

        except Exception as e:
            logger.error(f"Ошибка при обработке файла {filename}: {str(e)}")

logger.info("Обработка файлов завершена")

Выходные данные были обрезаны до нескольких последних строк (5000).
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once
    handle._run()
  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue
    await self.process_one()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 499, in process_one
    await dispatch(*args)
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 406, in dispatch_shell
    await resu